<a href="https://colab.research.google.com/github/robert-pineau/CIND-860-Capstone/blob/main/CIND860_image_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pylibjpeg
!pip install pydicom
!pip install pylibjpeg-libjpeg

In [ ]:
import os
import glob
import re
import cv2
import numpy as np
import pandas as pd
import libjpeg
import pylibjpeg
import pydicom as dicom
import skimage as ski
from pydicom.pixel_data_handlers.util import apply_voi_lut
from pydicom.pixel_data_handlers.util import apply_voi
from pydicom.pixel_data_handlers.util import apply_modality_lut
from pydicom.pixel_data_handlers.util import apply_windowing

In [ ]:
image_base_dir= "/mnt/wd/CIND860/database/train_images"
image_dir= "/mnt/wd/CIND860/database/square_cc_images"
dataset_file = "/mnt/wd/CIND860/train.csv"
used_type = "CC"

In [ ]:
#Based on a sampling of 5000 images after auto-croping
#the average aspect ratio was approximately 2.0:1
#For a neural network, was going to reduce image to 512x1024
#
#However, in order to use a predefined model, and their pre-trained weights
#need to make the images square, as more models use 224x224, or 227x227.
#
def lin_scale(img, new_w=1024, new_h=1024, colour='black'):
   aspect_ratio = img.shape[0]/img.shape[1]

   calc_h = int(new_w*(img.shape[0]/img.shape[1]))
   calc_w = int(new_h*(img.shape[1]/img.shape[0]))

   if colour == 'white':
      pad_colour = [255,255,255]
   else:
      pad_colour = [0,0,0]


   if calc_h > new_h:
      #calc height is larger than desired height.
      #therefore scale to x=calc_w, y=new_h
      #then pad right make x == new_w
      resize = cv2.resize(img, (calc_w,new_h))
      pad = new_w-calc_w
      resize2 = cv2.copyMakeBorder(resize,0,0,0,pad,cv2.BORDER_CONSTANT,None,pad_colour)

   else:
      #calc height is smaller than desired height.
      #therefore scale to x=new_w, y=calc_h
      #then pad top and bottom to make y == new_h
      resize = cv2.resize(img, (new_w,calc_h))

      pad = new_h-calc_h
      #Need to split pad up so we keep image centered top to bottom as cropped.
      #(ie pad half on bottom, half on top.
      pad1 = int(pad/2)
      pad2 = pad-pad1
      resize2 = cv2.copyMakeBorder(resize,pad1,pad2,0,0,cv2.BORDER_CONSTANT,None,pad_colour)

   return(resize2)


#Need to go through all the found images, count only view type 'CC', without implant, and cancerous.
#This number will be used to count a suitable balanced set of non cancerous images.
def get_cancer_count(image_list,used_type):
  cancer_count = 0

  for dcm_name in image_list:
    result = re.search(r"\/(\d+)\/(\d+)\.dcm", dcm_name)
    patient_id = int(result.group(1))
    image_id = int(result.group(2))

    my_index = np.where(df['image_id'] == image_id)[0]
    image_type = df.at[int(my_index),'view']
    implant = df.at[int(my_index),'implant']
    cancer = df.at[int(my_index),'cancer']

    if implant:
      continue

    if image_type != used_type:
      continue

    if cancer:
      cancer_count += 1

    return cancer_count



def process_image(image_dir,image_id,dcm_name):
  png_name = re.sub(r'\.dcm$', '.png', dcm_name)
  #print(f"{dcm_name}-->{png_name}:")

  #Read dicom(mammogram/xray file in)
  dcm = dicom.dcmread(dcm_name)

  #Extract into pixels.
  img = dcm.pixel_array.astype(np.float64)

  #apply conversion to greyscale.
  #only if the VOI LUT data is present in the dicom file.
  if [0x0028, 0x1056] in dcm:
    img = apply_voi_lut(img, dcm, index=0)

  #Rescale colour of each pixel into 8 bits
  img = (np.maximum(img, 0) / img.max()) * 255.0

  #Convert to uint8
  img = img.astype(np.uint8)

  #Convert from single channel colour into BGR format.
  img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

  #Some dicom files were type "MONOCHROME1" and some "MONOCHROME2"
  #The MONOCROME1 files are white based, whereas the MONOCHROME2 are the more customary black based.
  #Invert colours if dicom was type "MONOCHROME1"
  if dcm[0x0028, 0x0004].value == 'MONOCHROME1':
    img = 255-img

  #Calculate Aspect ratio of the original incoming file
  aspect_ratio_in = img.shape[0]/img.shape[1]

  #this returns 0 or 255 for every pixel if it is in the range between lower and upper.
  #for this purposes, we want to identify what part of the image is non black.
  threshold = cv2.inRange(img, lower, upper)

  #this finds contours within the image after the non black areas were identified.
  contours = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  contours = contours[0] if len(contours) == 2 else contours[1]

  #find the main contour in the image, basically the area with the most infomation.
  main_contour = max(contours, key=cv2.contourArea)

  # get a rectangle bounding the main contour.
  x,y,w,h = cv2.boundingRect(main_contour)

  #add some padding to the values. (50 pixels to top and bottom, and 150 pixels to sides.)
  #The min and max functions prevent going beyond the image's bounds.
  x1 = max(0,x-150)
  y1 = max(0,y-50)
  x2 = min(img.shape[1],x+w+150)
  y2 = min(img.shape[0],y+h+50)

  # crop the image at the bounds
  crop = img[y1:y2, x1:x2]

  if x > 100:
    #breast image is left facing, therefore want to flip it on the horizontal axis.
    crop = cv2.flip(crop,1)

  #Use out custom resize method.
  resize = lin_resize(crop,1024,1024,'black')

  #Write as a png file.
  cv2.imwrite(f"{image_dir}/{image_id}.png", resize)


In [ ]:
#For the automatic cropping of the image being performed below.
# set color bounds of white region
lower = (10,10,10) # lower bound for each channel(RGB)
upper = (255,255,255) # upper bound for each channel(RGB)


#Read in the database file for details regarding each image/patient.
df=pd.read_csv(dataset_file,sep=',')

image_list = glob.glob(os.path.join("", f"{image_base_dir}/*/*.dcm"))
random.shuffle(image_list)

cancer_count = get_cancer_count(image_list,used_type)

#We are looking for a balanced dataset, so get equal number of cacncerous, and non-cancerous images.
images_needed = cancer_count
with_cn = 0
without_cn = 0

for dcm_name in image_list:
  result = re.search(r"\/(\d+)\/(\d+)\.dcm", dcm_name)
  patient_id = int(result.group(1))
  image_id = int(result.group(2))

  my_index = np.where(df['image_id'] == image_id)[0]
  image_type = df.at[int(my_index),'view']
  implant = df.at[int(my_index),'implant']
  cancer = df.at[int(my_index),'cancer']

  #Skipping any with implants.
  if implant:
    continue

  if image_type != used_type:
    continue

  #Keep track of how many images for each of "With Cancer" and "Without Cancer" have been
  #converted and re-saved.
  #Once we get to the limit, stop processing files.
  if cancer and with_cn < images_needed:
    with_cn += 1
  elif not cancer and without_cn < images_needed:
    without_cn += 1
  else:
    continue

  #If we get this far we will use this image:
  print(f"Patient ID:***{patient_id}*** Image ID:***{image_id}*** Image Type:***{image_type}*** Implant: ***{implant}*** Cancer: ***{cancer}***")

  process_image(image_dir,image_id,dcm_name)

print(f"FOUND ***{with_cn+without_cn}*** images total")
print(f"FOUND ***{with_cn}*** with cancer")
